# Python で気軽に化学・化学工学
# 第 8 章 モデル y = f(x) を構築して、新たなサンプルの y を推定する
## 8.7.2 サポートベクター回帰 (Support Vector Regression, SVR)

## Jupyter Notebook の有用なショートカットのまとめ
- <kbd>Esc</kbd>: コマンドモードに移行（セルの枠が青）
- <kbd>Enter</kbd>: 編集モードに移行（セルの枠が緑）
- コマンドモードで <kbd>M</kbd>: Markdown セル (説明・メモを書く用) に変更
- コマンドモードで <kbd>Y</kbd>: Code セル (Python コードを書く用) に変更
- コマンドモードで <kbd>H</kbd>: ヘルプを表示
- コマンドモードで <kbd>A</kbd>: ひとつ**上**に空のセルを挿入
- コマンドモードで <kbd>B</kbd>: ひとつ**下**に空のセルを挿入
- コマンドモードで <kbd>D</kbd><kbd>D</kbd>: セルを削除
- <kbd>Ctrl</kbd>+<kbd>Enter</kbd>: セルの内容を実行
- <kbd>Shift</kbd>+<kbd>Enter</kbd>: セルの内容を実行して下へ

わからないことがありましたら、関係する単語やエラーの文章などでウェブ検索してご自身で調べてみましょう。

### 沸点のデータセット (descriptors_8_with_boiling_point.csv)
Hall and Story が収集した[沸点のデータセット](https://pubs.acs.org/doi/abs/10.1021/ci960375x)。294 個の化合物について、沸点 (Boiling Point) が測定されており、8 つの分子記述子 (特徴量) で化学構造が数値化されています。特徴量は、分子量 (MolWt)、水素原子以外の原子で計算された分子量 (HeavyAtomMolWt)、価電子の数 (NumValenceElectrons)、水素原子以外の原子の数 (HeavyAtomCount)、窒素原子と酸素原子の数 (NOCount)、水素原子と炭素原子以外の原子の数 (NumHeteroatoms)、回転可能な結合の数 (NumRotatableBonds)、環の数 (RingCount) です。

In [ ]:
import pandas as pd # pandas のインポート

In [ ]:
dataset = pd.read_csv('descriptors_8_with_boiling_point.csv', index_col=0, header=0) # 沸点のデータセットの読み込み

In [ ]:
x = dataset.iloc[:, 1:] # 記述子を 説明変数 x とします

In [ ]:
y = dataset.iloc[:, 0] # 沸点を目的変数 y とします

トレーニングデータとテストデータの分割

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# ランダムにトレーニングデータとテストデータとに分割。random_state に数字を与えることで、別のときに同じ数字を使えば、ランダムとはいえ同じ結果にすることができます
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=94, shuffle=True, random_state=99)

特徴量の標準化 (オートスケーリング)

In [ ]:
autoscaled_x_train = (x_train - x_train.mean()) / x_train.std() # トレーニングデータの説明変数の標準化。平均を引いてから、標準偏差で割ります

In [ ]:
autoscaled_x_test = (x_test - x_train.mean()) / x_train.std() # テストデータの説明変数の標準化には、トレーニングデータの平均と標準偏差を用いることに注意してください

In [ ]:
autoscaled_y_train = (y_train - y_train.mean()) / y_train.std() # トレーニングデータの目的変数の標準化

サポートベクター回帰 (Support Vector Regression, SVR) の実行

ガウシアンカーネルを用いた SVR

In [ ]:
from sklearn.svm import SVR # SVR の実行に使用

In [ ]:
model = SVR(kernel='rbf', C=1, epsilon=1, gamma=1) # SVRモデルの宣言

In [ ]:
model.fit(autoscaled_x_train, autoscaled_y_train) # SVRモデル構築

サポートベクターとその係数 (*α*U - *α*L)、 および定数項 (*u*) の確認

In [ ]:
model.support_

In [ ]:
len(model.support_)

In [ ]:
model.dual_coef_

In [ ]:
model.intercept_

トレーニングデータの y の値の推定

In [ ]:
estimated_y_train = pd.DataFrame(model.predict(autoscaled_x_train)) # トレーニングデータの y の値を推定し、pandas の DataFrame 型に変換

In [ ]:
estimated_y_train = estimated_y_train * y_train.std() + y_train.mean() # スケールをもとに戻します

In [ ]:
estimated_y_train.index = x_train.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_train.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_train # 念のため確認

In [ ]:
estimated_y_train.to_csv('estimated_y_train.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

トレーニングデータの y の実測値 vs. 推定値プロット

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.figure as figure # 図の調整に使用

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y_train, estimated_y_train.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y_train.max(), estimated_y_train.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y_train.min(), estimated_y_train.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

トレーニングデータの r<sup>2</sup>, MAE

In [ ]:
from sklearn import metrics

In [ ]:
metrics.r2_score(y_train, estimated_y_train) # r2

In [ ]:
metrics.mean_absolute_error(y_train, estimated_y_train) # MAE

テストデータの y の値の推定。トレーニングデータをテストデータに変えるだけで、実行する内容はトレーニングデータのときと同じです

In [ ]:
estimated_y_test = pd.DataFrame(model.predict(autoscaled_x_test)) # テストデータの y の値を推定し、pandas の DataFrame 型に変換

In [ ]:
estimated_y_test = estimated_y_test * y_train.std() + y_train.mean() # スケールをもとに戻します

In [ ]:
estimated_y_test.index = x_test.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_test.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_test # 念のため確認

In [ ]:
estimated_y_test.to_csv('estimated_y_test.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

テストデータの y の実測値 vs. 推定値プロット

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y_test, estimated_y_test.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y_test.max(), estimated_y_test.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y_test.min(), estimated_y_test.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

テストデータの r<sup>2</sup>, MAE

In [ ]:
metrics.r2_score(y_test, estimated_y_test) # r2

In [ ]:
metrics.mean_absolute_error(y_test, estimated_y_test) # MAE

クロスバリデーションによる *C*, *ε*, *γ* の最適化

In [ ]:
import numpy as np # NumPy のインポート

In [ ]:
nonlinear_svr_cs = 2 ** np.arange(-5, 11, 1.0) # C の候補

In [ ]:
nonlinear_svr_cs # 念のため確認

In [ ]:
nonlinear_svr_epsilons = 2 ** np.arange(-10, 1, 1.0) # εの候補

In [ ]:
nonlinear_svr_epsilons # 念のため確認

In [ ]:
nonlinear_svr_gammas = 2 ** np.arange(-20, 11, dtype=float) # γの候補

In [ ]:
nonlinear_svr_gammas # 念のため確認

グラム行列の分散の最大化によるガウシアンカーネルにおける *γ* の最適化

In [ ]:
from scipy.spatial.distance import cdist # トレーニングデータにおけるユークリッド距離に使用

In [ ]:
square_of_euclidean_distance = cdist(autoscaled_x_train, autoscaled_x_train, metric='sqeuclidean')

In [ ]:
square_of_euclidean_distance # 念のため確認

In [ ]:
np.exp(- 2 * square_of_euclidean_distance) # γ=2 のときの、ガウシアンカーネルのグラム行列

In [ ]:
variance_of_gram_matrix = [] # この変数にグラム行列の分散を入れていきます

In [ ]:
for nonlinear_svr_gamma in nonlinear_svr_gammas:
    gram_matrix = np.exp(- nonlinear_svr_gamma * square_of_euclidean_distance)
    variance_of_gram_matrix.append(gram_matrix.var(ddof=1))

In [ ]:
optimal_nonlinear_svr_gamma = nonlinear_svr_gammas[variance_of_gram_matrix.index(max(variance_of_gram_matrix))] # グラム行列の分散が最大となるγ

In [ ]:
optimal_nonlinear_svr_gamma # 念のため確認

クロスバリデーションの設定

In [ ]:
fold_number = 5 # クロスバリデーションのfold数

In [ ]:
from sklearn.model_selection import KFold # クロスバリデーションの分割の設定に使用

In [ ]:
fold = KFold(n_splits=fold_number, shuffle=True, random_state=9) # クロスバリデーションの分割の設定

In [ ]:
from sklearn.model_selection import GridSearchCV # グリッドサーチに使用

クロスバリデーションによる *ε* の最適化

In [ ]:
model_for_cross_validation = SVR(kernel='rbf', C=3, gamma=optimal_nonlinear_svr_gamma)

In [ ]:
gs_cv = GridSearchCV(model_for_cross_validation, {'epsilon': nonlinear_svr_epsilons}, cv=fold)

In [ ]:
gs_cv.fit(autoscaled_x_train, autoscaled_y_train)

In [ ]:
optimal_nonlinear_svr_epsilon = gs_cv.best_params_['epsilon']

In [ ]:
optimal_nonlinear_svr_epsilon # 念のため確認

クロスバリデーションによる *C* の最適化

In [ ]:
model_for_cross_validation = SVR(kernel='rbf', epsilon=optimal_nonlinear_svr_epsilon, gamma=optimal_nonlinear_svr_gamma)

In [ ]:
gs_cv = GridSearchCV(model_for_cross_validation, {'C': nonlinear_svr_cs}, cv=fold)

In [ ]:
gs_cv.fit(autoscaled_x_train, autoscaled_y_train)

In [ ]:
optimal_nonlinear_svr_c = gs_cv.best_params_['C']

In [ ]:
optimal_nonlinear_svr_c # 念のため確認

クロスバリデーションによる *γ* の最適化

In [ ]:
model_for_cross_validation = SVR(kernel='rbf', epsilon=optimal_nonlinear_svr_epsilon, C=optimal_nonlinear_svr_c)

In [ ]:
gs_cv = GridSearchCV(model_for_cross_validation, {'gamma': nonlinear_svr_gammas}, cv=fold)

In [ ]:
gs_cv.fit(autoscaled_x_train, autoscaled_y_train)

In [ ]:
optimal_nonlinear_svr_gamma = gs_cv.best_params_['gamma']

In [ ]:
optimal_nonlinear_svr_gamma # 念のため確認

SVR モデルの構築および予測

In [ ]:
model = SVR(kernel='rbf', C=optimal_nonlinear_svr_c, epsilon=optimal_nonlinear_svr_epsilon, gamma=optimal_nonlinear_svr_gamma) # SVRモデルの宣言

In [ ]:
model.fit(autoscaled_x_train, autoscaled_y_train) # SVRモデル構築

サポートベクターとその係数 (*α*U - *α*L)、 および定数項 (*u*) の確認

In [ ]:
model.support_

In [ ]:
len(model.support_)

In [ ]:
model.dual_coef_

In [ ]:
model.intercept_

トレーニングデータの y の値の推定

In [ ]:
estimated_y_train = pd.DataFrame(model.predict(autoscaled_x_train)) # トレーニングデータの y の値を推定し、pandas の DataFrame 型に変換

In [ ]:
estimated_y_train = estimated_y_train * y_train.std() + y_train.mean() # スケールをもとに戻します

In [ ]:
estimated_y_train.index = x_train.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_train.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_train # 念のため確認

In [ ]:
estimated_y_train.to_csv('estimated_y_train.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

トレーニングデータの y の実測値 vs. 推定値プロット

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.figure as figure # 図の調整に使用

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y_train, estimated_y_train.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y_train.max(), estimated_y_train.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y_train.min(), estimated_y_train.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

トレーニングデータの r<sup>2</sup>, MAE

In [ ]:
from sklearn import metrics

In [ ]:
metrics.r2_score(y_train, estimated_y_train) # r2

In [ ]:
metrics.mean_absolute_error(y_train, estimated_y_train) # MAE

テストデータの y の値の推定。トレーニングデータをテストデータに変えるだけで、実行する内容はトレーニングデータのときと同じです

In [ ]:
estimated_y_test = pd.DataFrame(model.predict(autoscaled_x_test)) # テストデータの y の値を推定し、pandas の DataFrame 型に変換

In [ ]:
estimated_y_test = estimated_y_test * y_train.std() + y_train.mean() # スケールをもとに戻します

In [ ]:
estimated_y_test.index = x_test.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_test.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_test # 念のため確認

In [ ]:
estimated_y_test.to_csv('estimated_y_test.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

テストデータの y の実測値 vs. 推定値プロット

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y_test, estimated_y_test.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y_test.max(), estimated_y_test.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y_test.min(), estimated_y_test.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

テストデータの r<sup>2</sup>, MAE

In [ ]:
metrics.r2_score(y_test, estimated_y_test) # r2

In [ ]:
metrics.mean_absolute_error(y_test, estimated_y_test) # MAE

自分のデータセットをお持ちの方は、そのデータセットでも今回の内容を確認してみましょう。

線形カーネルを用いた SVR

In [ ]:
from sklearn.svm import SVR # SVR の実行に使用

In [ ]:
model = SVR(kernel='linear', C=1, epsilon=1) # SVRモデルの宣言

In [ ]:
model.fit(autoscaled_x_train, autoscaled_y_train) # SVRモデル構築

サポートベクターとその係数 (*α*U - *α*L)、 および定数項 (*u*) の確認

In [ ]:
model.support_

In [ ]:
len(model.support_)

In [ ]:
model.dual_coef_

In [ ]:
model.intercept_

トレーニングデータの y の値の推定

In [ ]:
estimated_y_train = pd.DataFrame(model.predict(autoscaled_x_train)) # トレーニングデータの y の値を推定し、pandas の DataFrame 型に変換

In [ ]:
estimated_y_train = estimated_y_train * y_train.std() + y_train.mean() # スケールをもとに戻します

In [ ]:
estimated_y_train.index = x_train.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_train.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_train # 念のため確認

In [ ]:
estimated_y_train.to_csv('estimated_y_train.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

トレーニングデータの y の実測値 vs. 推定値プロット

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.figure as figure # 図の調整に使用

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y_train, estimated_y_train.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y_train.max(), estimated_y_train.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y_train.min(), estimated_y_train.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

トレーニングデータの r<sup>2</sup>, MAE

In [ ]:
from sklearn import metrics

In [ ]:
metrics.r2_score(y_train, estimated_y_train) # r2

In [ ]:
metrics.mean_absolute_error(y_train, estimated_y_train) # MAE

テストデータの y の値の推定。トレーニングデータをテストデータに変えるだけで、実行する内容はトレーニングデータのときと同じです

In [ ]:
estimated_y_test = pd.DataFrame(model.predict(autoscaled_x_test)) # テストデータの y の値を推定し、pandas の DataFrame 型に変換

In [ ]:
estimated_y_test = estimated_y_test * y_train.std() + y_train.mean() # スケールをもとに戻します

In [ ]:
estimated_y_test.index = x_test.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_test.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_test # 念のため確認

In [ ]:
estimated_y_test.to_csv('estimated_y_test.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

テストデータの y の実測値 vs. 推定値プロット

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y_test, estimated_y_test.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y_test.max(), estimated_y_test.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y_test.min(), estimated_y_test.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

テストデータの r<sup>2</sup>, MAE

In [ ]:
metrics.r2_score(y_test, estimated_y_test) # r2

In [ ]:
metrics.mean_absolute_error(y_test, estimated_y_test) # MAE

クロスバリデーションによる *C* と *ε* の最適化

In [ ]:
import numpy as np # NumPy のインポート

In [ ]:
linear_svr_cs = 2 ** np.arange(-5, 9, 1.0) # C の候補

In [ ]:
linear_svr_cs # 念のため確認

In [ ]:
linear_svr_epsilons = 2 ** np.arange(-10, 1, 1.0) # εの候補

In [ ]:
linear_svr_epsilons # 念のため確認

クロスバリデーションの設定

In [ ]:
fold_number = 5 # クロスバリデーションのfold数

In [ ]:
from sklearn.model_selection import KFold # クロスバリデーションの分割の設定に使用

In [ ]:
fold = KFold(n_splits=fold_number, shuffle=True, random_state=9) # クロスバリデーションの分割の設定

In [ ]:
model_for_cross_validation = SVR(kernel='linear')

グリッドサーチの設定

In [ ]:
from sklearn.model_selection import GridSearchCV # グリッドサーチに使用

In [ ]:
gs_cv = GridSearchCV(model_for_cross_validation, {'C':linear_svr_cs, 'epsilon':linear_svr_epsilons}, cv=fold) # クロスバリデーションによるグリッドサーチの設定

クロスバリデーションによるグリッドサーチの実行

In [ ]:
gs_cv.fit(autoscaled_x_train, autoscaled_y_train)

In [ ]:
optimal_linear_c = gs_cv.best_params_['C']
optimal_linear_epsilon = gs_cv.best_params_['epsilon']

In [ ]:
optimal_linear_c # 念のため確認

In [ ]:
optimal_linear_epsilon # 念のため確認

SVR モデルの構築および予測

In [ ]:
model = SVR(kernel='linear', C=optimal_linear_c, epsilon=optimal_linear_epsilon) # SVRモデルの宣言

In [ ]:
model.fit(autoscaled_x_train, autoscaled_y_train) # SVRモデル構築

サポートベクターとその係数 (*α*U - *α*L)、 および定数項 (*u*) の確認

In [ ]:
model.support_

In [ ]:
len(model.support_)

In [ ]:
model.dual_coef_

In [ ]:
model.intercept_

トレーニングデータの y の値の推定

In [ ]:
estimated_y_train = pd.DataFrame(model.predict(autoscaled_x_train)) # トレーニングデータの y の値を推定し、pandas の DataFrame 型に変換

In [ ]:
estimated_y_train = estimated_y_train * y_train.std() + y_train.mean() # スケールをもとに戻します

In [ ]:
estimated_y_train.index = x_train.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_train.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_train # 念のため確認

In [ ]:
estimated_y_train.to_csv('estimated_y_train.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

トレーニングデータの y の実測値 vs. 推定値プロット

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.figure as figure # 図の調整に使用

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y_train, estimated_y_train.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y_train.max(), estimated_y_train.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y_train.min(), estimated_y_train.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

トレーニングデータの r<sup>2</sup>, MAE

In [ ]:
from sklearn import metrics

In [ ]:
metrics.r2_score(y_train, estimated_y_train) # r2

In [ ]:
metrics.mean_absolute_error(y_train, estimated_y_train) # MAE

テストデータの y の値の推定。トレーニングデータをテストデータに変えるだけで、実行する内容はトレーニングデータのときと同じです

In [ ]:
estimated_y_test = pd.DataFrame(model.predict(autoscaled_x_test)) # テストデータの y の値を推定し、pandas の DataFrame 型に変換

In [ ]:
estimated_y_test = estimated_y_test * y_train.std() + y_train.mean() # スケールをもとに戻します

In [ ]:
estimated_y_test.index = x_test.index # サンプル名を、元のデータのサンプル名に

In [ ]:
estimated_y_test.columns = ['estimated_y'] # 列名を変更

In [ ]:
estimated_y_test # 念のため確認

In [ ]:
estimated_y_test.to_csv('estimated_y_test.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

テストデータの y の実測値 vs. 推定値プロット

In [ ]:
plt.rcParams['font.size'] = 18 # 横軸や縦軸の名前の文字などのフォントのサイズ
plt.figure(figsize=figure.figaspect(1)) # 図の形を正方形に
plt.scatter(y_test, estimated_y_test.iloc[:, 0]) # 散布図。estimated_y_train は 200×1 の行列のため、0 列目を選択する必要があります
y_max = max(y_test.max(), estimated_y_test.iloc[:, 0].max()) # 実測値の最大値と、推定値の最大値の中で、より大きい値を取得
y_min = min(y_test.min(), estimated_y_test.iloc[:, 0].min()) # 実測値の最小値と、推定値の最小値の中で、より小さい値を取得
plt.plot([y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)], 'k-') # 取得した最小値-5%から最大値+5%まで、対角線を作成
plt.ylim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # y 軸の範囲の設定
plt.xlim(y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)) # x 軸の範囲の設定 
plt.xlabel("actual y") # x 軸の名前
plt.ylabel("estimated y") # y 軸の名前
plt.show() # 以上の設定で描画

テストデータの r<sup>2</sup>, MAE

In [ ]:
metrics.r2_score(y_test, estimated_y_test) # r2

In [ ]:
metrics.mean_absolute_error(y_test, estimated_y_test) # MAE